In [2]:
# Install required packages.
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-geometric

     |████████████████████████████████| 2.6MB 2.6MB/s 
     |████████████████████████████████| 1.5MB 2.4MB/s 
     |████████████████████████████████| 215kB 3.9MB/s 
     |████████████████████████████████| 235kB 21.6MB/s 
     |████████████████████████████████| 2.2MB 37.9MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 


In [3]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch
from torch.nn import Linear
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.utils import to_dense_adj, dense_to_sparse, add_self_loops, degree
from torch_geometric.nn import MessagePassing

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
#print(torch.cuda.get_device_name(0))

def visualize(h, color):
    z = TSNE(n_components=2).fit_transform(out.detach().cpu().numpy())

    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])

    plt.scatter(z[:, 0], z[:, 1], s=70, c=color, cmap="Set2")
    plt.show()

from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

dataset = Planetoid(root='data/', name='Cora', transform=NormalizeFeatures())

print()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
print(f'Contains self-loops: {data.contains_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

cuda
Processing...
Done!

Dataset: Cora():
Number of graphs: 1
Number of features: 1433
Number of classes: 7

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])
Number of nodes: 2708
Number of edges: 10556
Average node degree: 3.90
Number of training nodes: 140
Training node label rate: 0.05
Contains isolated nodes: False
Contains self-loops: False
Is undirected: True


In [4]:
import numpy as np

In [43]:
beta=2
alpha=0.2
dataset = Planetoid(root='data/', name='Cora', transform=NormalizeFeatures())

print()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
print(f'Contains self-loops: {data.contains_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')
class FGCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(FGCNConv, self).__init__(aggr='add')  # "Add" aggregation (Step 5).
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index, edge_attr):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        
        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)

        # Step 3: Compute normalization.
        row, col = edge_index
        deg_row = degree(row, x.size(0), dtype=x.dtype)
        deg_col = degree(col, x.size(0), dtype=x.dtype)
        deg_row_inv_sqrt = deg_row.pow(-0.5)
        deg_col_inv_sqrt = deg_col.pow(-0.5)
        norm = deg_row_inv_sqrt[row] * deg_col_inv_sqrt[col]
        norm = norm * edge_attr

        # Step 4-5: Start propagating messages.
        return self.propagate(edge_index, x=x, norm=norm)

    def message(self, x_j, norm):
        # x_j has shape [E, out_channels]

        # Step 4: Normalize node features.
        return norm.view(-1, 1) * x_j


def PowerInd(edge_index,beta):
    # returns the edge list in COO format with shape [2,num_edges,num_2_hop neighbors]
    # the edge weighted is stored in the edge_attr, 
    edge_index=data.edge_index
    
    adj = to_dense_adj(edge_index)
    adj = torch.reshape(adj,(adj.size(1),adj.size(2)))
    adj = torch.matmul(torch.matmul(torch.diag(1/adj.sum(axis=1)),adj),torch.diag(1/adj.sum(axis=1)))
    print(adj.shape)
    adj_be=adj-beta*torch.eye(data.num_nodes)

    new_adj = torch.eye(data.num_nodes)-alpha*torch.matmul(adj_be,adj_be)
    edge_index_f, edge_attr_f = dense_to_sparse(new_adj)
    
    return edge_index_f, edge_attr_f


edge_index_f, edge_attr_f = PowerInd(data.edge_index,beta)

data_f = Data(edge_index=edge_index_f, x= data.x,edge_attr=edge_attr_f, test_mask=data.test_mask,train_mask=data.train_mask,y=data.y)


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, dataset.num_classes)

        self.fconv1 = FGCNConv(dataset.num_features, hidden_channels)       
        self.fconv2 = FGCNConv(hidden_channels, dataset.num_classes)
        self.w = nn.Parameter(torch.rand(1,1,requires_grad=True))
        self.w0 = nn.Parameter(torch.rand(1,1,requires_grad=True))
        #self.lin1 = Linear(dataset.num_classes,dataset.num_classes)

    def forward(self, x, edge_index, edge_index_f, edge_attr_f):
        uf = self.conv1(x, edge_index)
        uf = uf.relu()
        #uf = F.dropout(uf, p=0.5, training=self.training)
        uf = self.conv2(uf, edge_index)
        

        f = self.fconv1(x, edge_index_f, edge_attr_f)
        f = f.relu()
        #f = F.dropout(f, p=0.5, training=self.training)
        f = self.fconv2(f, edge_index_f, edge_attr_f)
        
        uf = f*self.w0 +uf*self.w
        #print(self.w)
        #uf = torch.cat((uf,f),dim=1)
        #uf = uf.relu()
        #uf = self.lin1(uf)

        return uf
#out = model(data.x, data.edge_index)
#visualize(out, color=data.y)

#from IPython.display import Javascript  # Restrict height of output cell.
#display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=128).to(device)
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    optimizer.zero_grad()  # Clear gradients.
    out = model(data.x, data.edge_index, data_f.edge_index, data_f.edge_attr)  # Perform a single forward pass.
    loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    return loss

def test():
    model.eval()
    out = model(data.x, data.edge_index, data_f.edge_index, data_f.edge_attr)
    pred = out.argmax(dim=1)  # Use the class with highest probability.
    test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
    test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.
    return test_acc

data = data.to(device)
data_f = data_f.to(device)
a_list=[]
for epoch in range(1, 1000):
    loss = train()
    if epoch%100==0 or epoch<10:
        test_acc = test()
        a_list.append(test_acc)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Test Acc:{test_acc:.4f}')


test_acc = test()
print(f'Test Accuracy: {test_acc:.4f}')


Dataset: Cora():
Number of graphs: 1
Number of features: 1433
Number of classes: 7

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])
Number of nodes: 2708
Number of edges: 10556
Average node degree: 3.90
Number of training nodes: 140
Training node label rate: 0.05
Contains isolated nodes: False
Contains self-loops: False
Is undirected: True
torch.Size([2708, 2708])
GCN(
  (conv1): GCNConv(1433, 128)
  (conv2): GCNConv(128, 7)
  (fconv1): FGCNConv(
    (lin): Linear(in_features=1433, out_features=128, bias=True)
  )
  (fconv2): FGCNConv(
    (lin): Linear(in_features=128, out_features=7, bias=True)
  )
)
Epoch: 001, Loss: 1.9457, Test Acc:0.2480
Epoch: 002, Loss: 1.9423, Test Acc:0.1970
Epoch: 003, Loss: 1.9386, Test Acc:0.2010
Epoch: 004, Loss: 1.9339, Test Acc:0.1940
Epoch: 005, Loss: 1.9284, Test Acc:0.1820
Epoch: 006, Loss: 1.9218, Test Acc:0.1620
Epoch: 007, Loss: 1.9141, Test Acc:0.1510
Epoch: 008, Loss: 1.9050, Test Acc:

KeyboardInterrupt: ignored

In [47]:
edge_index=data.edge_index

adj = to_dense_adj(edge_index)
adj = torch.reshape(adj,(adj.size(1),adj.size(2)))
adj = torch.matmul(torch.matmul(torch.diag(1/adj.sum(axis=1)),adj),torch.diag(1/adj.sum(axis=1)))
u,v=torch.eig(adj)
adj_be=adj-beta*torch.eye(data.num_nodes)

new_adj = torch.eye(data.num_nodes)-alpha*torch.matmul(adj_be,adj_be)
edge_index_f, edge_attr_f = dense_to_sparse(new_adj)

RuntimeError: ignored

In [48]:
u

tensor([[ 1.0000,  0.0000],
        [-1.0000,  0.0000],
        [ 1.0000,  0.0000],
        ...,
        [-0.3904,  0.0000],
        [ 1.0000,  0.0000],
        [-1.0000,  0.0000]], device='cuda:0')

In [13]:
edge_index=data.edge_index
row, col = edge_index

adj = to_dense_adj(edge_index)
adj = torch.reshape(adj,(adj.size(1),adj.size(2)))
adj_be=adj-beta*torch.eys(data.num_nodes)
new_adj = torch.eys(data.num_nodes)-alpha*torch.matmul(adj_be,adj_be)
edge_index_f, edge_attr_f = dense_to_sparse(new_adj)
return edge_index, edge_attr

tensor([[ 14.3909,   0.0000],
        [-12.3658,   0.0000],
        [ 11.6385,   0.0000],
        ...,
        [ -0.6180,   0.0000],
        [ -1.0000,   0.0000],
        [  1.0000,   0.0000]])
torch.Size([2708, 2708])
tensor([3.6400, 1.0000, 1.0000,  ..., 1.0000, 2.0000, 4.6400])


In [ ]:
def PowerInd(edge_index,beta):
    # returns the edge list in COO format with shape [2,num_edges,num_2_hop neighbors]
    # the edge weighted is stored in the edge_attr, 
    row, col = edge_index
    edge_attr = torch.ones(edge_index.size(1))
    edge_attr = torch.mul(edge_attr,-2*beta)
    adj = to_dense_adj(edge_index)
    adj = torch.reshape(adj,(adj.size(1),adj.size(2)))
    adj = torch.matmul(adj,adj)
    adj = adj+torch.mul(torch.eye(adj.size(0),dtype=torch.float),beta*beta)
    edge_index_f, edge_attr_f = dense_to_sparse(adj)
    print(edge_attr_f)
    edge_index = torch.cat((edge_index,edge_index_f),dim=1)
    edge_attr = torch.cat((edge_attr,edge_attr_f),dim=0)
    return edge_index, edge_attr


edge_index_f, edge_attr_f = PowerInd(data.edge_index,beta)

data_f = Data(edge_index=edge_index_f, x= data.x,edge_attr=edge_attr_f, test_mask=data.test_mask,train_mask=data.train_mask,y=data.y)
